In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
def getGraphForRun(s):
    for i in s.split("\n"):
        print("'"+i+"'", end = " ")
    return s.split("\n")

In [6]:
graphs =  """fat200
GSE1730_q
GSE10158_q
orani
ego-facebook
grqc_q
uc64
government
wiki-elec
lastfm
hepPh
astroPh
condmat
Enron_q
fb-pages
brightkite
livemocha
gowalla
citeseer
com-dblp
amazon_q
youtube_q
hyves_q
skitter
flixster
patent_q
soc-livejournal
dblp-author"""

In [7]:
graph_names = getGraphForRun(graphs)

'fat200' 'GSE1730_q' 'GSE10158_q' 'orani' 'ego-facebook' 'grqc_q' 'uc64' 'government' 'wiki-elec' 'lastfm' 'hepPh' 'astroPh' 'condmat' 'Enron_q' 'fb-pages' 'brightkite' 'livemocha' 'gowalla' 'citeseer' 'com-dblp' 'amazon_q' 'youtube_q' 'hyves_q' 'skitter' 'flixster' 'patent_q' 'soc-livejournal' 'dblp-author' 

In [8]:
def CPUArgs(path,graph_names,minmax = 0):
    df = pd.read_csv(path)
    qid = []
    for graph_name in graph_names:
        l = []
        temp = df[df['file_name']==graph_name]
        for core_value in [6,9,12,15,18,21,24]:
            vert = list(map(int,temp["vertices_with_coreValue_close_to_"+str(core_value)].values[0].split(" ")))
            degree = list(map(int,temp['Degree of vertices_with_coreValue_close_to_'+str(core_value)].values[0].split(" ")))
            if minmax==0:
                l.append(vert[np.argmax(degree)])
            else:
                l.append(vert[np.argmin(degree)])
        qid.append(" ".join(list(map(str,l))))
    return qid

In [9]:
cpu_args = CPUArgs("../../graphStatisticsFinal.csv",graph_names)

In [10]:
cpu_args

['1 1 1 1 1 1 1', '14 41 59 255 123 123 214', '185 57 168 167 167 61 61']

In [10]:
def GpuClientArgs(path,graph_names,output_dir,minmax=0,postfix="1 1"):
    df = pd.read_csv(path)
    for graph_name in graph_names:
        query_data = ''
        temp = df[df['file_name']==graph_name]
        for N1,N2 in zip([3,6,9,12,15,18,21],[6,9,12,15,18,21,24]):
            vert = list(map(int,temp["vertices_with_coreValue_close_to_"+str(N2)].values[0].split(" ")))
            degree = list(map(int,temp['Degree of vertices_with_coreValue_close_to_'+str(N2)].values[0].split(" ")))
            qid = vert[np.argmax(degree)] if minmax == 0 else vert[np.argmin(degree)]
            query = f"{N1} {N2} {qid} {postfix}\n"
            query_data +=query
        query_data+="server_exit"
        with open(os.path.join(output_dir,graph_name),'w') as f:
            f.write(query_data)
        print("completed: ",graph_name)
    print("Done yay! :)")

In [14]:
x = GpuClientArgs("../../graphStatisticsFinal.csv",graph_names,"../client/query/maxdeg",minmax= 0 )

completed:  fat200
completed:  GSE1730_q
completed:  GSE10158_q
completed:  orani
completed:  ego-facebook
completed:  grqc_q
completed:  uc64
completed:  government
completed:  wiki-elec
completed:  lastfm
completed:  hepPh
completed:  astroPh
completed:  condmat
completed:  Enron_q
completed:  fb-pages
completed:  brightkite
completed:  livemocha
completed:  gowalla
completed:  citeseer
completed:  com-dblp
completed:  amazon_q
completed:  youtube_q
completed:  hyves_q
completed:  skitter
completed:  flixster
completed:  patent_q
completed:  soc-livejournal
completed:  dblp-author
Done yay! :)


In [16]:
x = GpuClientArgs("../../graphStatisticsFinal.csv",graph_names,"../client/query/mindeg",minmax = 1)

completed:  fat200
completed:  GSE1730_q
completed:  GSE10158_q
completed:  orani
completed:  ego-facebook
completed:  grqc_q
completed:  uc64
completed:  government
completed:  wiki-elec
completed:  lastfm
completed:  hepPh
completed:  astroPh
completed:  condmat
completed:  Enron_q
completed:  fb-pages
completed:  brightkite
completed:  livemocha
completed:  gowalla
completed:  citeseer
completed:  com-dblp
completed:  amazon_q
completed:  youtube_q
completed:  hyves_q
completed:  skitter
completed:  flixster
completed:  patent_q
completed:  soc-livejournal
completed:  dblp-author
Done yay! :)
